In [1]:
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

In [3]:
RESULTS_FILE = "out/results/GAT_2024-05-23-16-24-07_results.h5ad"  # No-Graph model
# RESULTS_FILE = "out/results/GAT_2024-05-23-16-23-45_results.h5ad"  # GAT model
FOLD = 4
ddata = ad.read_h5ad(RESULTS_FILE.replace(".h5ad", f"_donor_{FOLD}.h5ad"))
ddata.uns["perf_test"]

{'accuracy': 0.7777777777777778,
 'f1': 0.8000000000000002,
 'precision': 0.8,
 'recall': 0.8,
 'roc_auc': 0.8999999999999999}

In [4]:
wang_labels = []
for i, donor in enumerate(ddata.obs_names):
    dddata = ddata[ddata.obs_names == donor].copy().obs
    cogdx = dddata["cogdx"].values[0]
    braaksc = dddata["braaksc"].values[0]
    ceradsc = dddata["ceradsc"].values[0]
    if cogdx == 4 and braaksc >= 4 and ceradsc <= 2:
        wang_labels.append("AD")
    elif cogdx == 1 and braaksc <= 3 and ceradsc >= 3:
        wang_labels.append("CT")
    else:
        wang_labels.append("Other")
ddata.obs["wang_label"] = wang_labels

ddata.obs["reagan_desc"] = ddata.obs["niareagansc"].map({4: "No AD", 3: "Low", 2: "Int", 1: "High"})
ddata.obs["reagan_dich"] = ddata.obs["niareagansc"].map({4: 0, 3: 0, 2: 1, 1: 1})

ddata.obs["cogdx_desc"] = ddata.obs["cogdx"].map({1: "NCI", 2: "MCI", 3: "MCI+", 4: "AD", 5: "AD+", 6: "Other"})
ddata.obs["cogdx_dich"] = ddata.obs["cogdx"].map({1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 0})

ddata.obs["y_true"] = ddata.obs["wang_label"].map({"AD": 1, "CT": 0, "Other": 2})
ddata.obs["acc"] = 1 - np.abs(ddata.obs["cogdx_dich"] - ddata.obs["y_pred"])

ddata.obs["sex"] = ddata.obs["msex"].map({0: "Female", 1: "Male"})

In [11]:
# Now we make tables for the disagreement between the different labels

ddata.obs["cogdx_simp"] = ddata.obs["cogdx_desc"].map(
    {"NCI": "NCI", "MCI": "MCI(+)", "MCI+": "MCI(+)", "AD": "AD(+)", "AD+": "AD(+)", "Other": "Other"}
)

# Wang vs Reagan
wang_reagan = pd.crosstab(ddata.obs["reagan_desc"], ddata.obs["cogdx_simp"])
# wang_reagan = wang_reagan.div(wang_reagan.sum(axis=1), axis=0)
display(wang_reagan)

cogdx_simp,AD(+),MCI(+),NCI,Other
reagan_desc,,,,
High,47,8,5,2
Int,52,35,38,2
Low,25,38,76,5
No AD,0,1,5,0
